In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules
import warnings

In [43]:
df = pd.read_csv('clean_class.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['month'] = df['Date'].dt.month.astype(int)
df['month'] = df['month'].apply(lambda x: list(calendar.month_abbr)[x])

comp_df = df.copy()
comp_df['covid'] = df['Date'].apply(lambda x: 
    'pre_covid' if x < pd.to_datetime("2020-03-01") 
    else ('post_covid' if x > pd.to_datetime("2020-06-01") else 'during_covid'))
#comp_df['victims'] = df['Victimisations'].apply(lambda x: 'victims more than 5 people' if x > 5 else 'victims less than 5 people')
comp_df

,Date,Victimisations,SEX,Age.Group.5Yr.Band,OOI.Exclusion,Ethnic.Group,ANZSOC.Group,class,month,covid
0,2015-06-01,2,Female,Middle-aged-young,Court action,Maori,"Theft (Except Motor Vehicles), N.E.C.",0,Jun,pre_covid
1,2015-06-01,1,Male,Middle-aged-young,Court action,European,"Theft (Except Motor Vehicles), N.E.C.",0,Jun,pre_covid
2,2015-06-01,2,Male,Middle-aged-old,Court action,Pacific Island,Serious Assault Resulting in Injury,0,Jun,pre_covid
3,2015-06-01,1,Male,Elderly,Court action,European,Illegal Use of a Motor Vehicle,0,Jun,pre_covid
4,2015-06-01,2,Male,Middle-aged-young,Court action,European,Aggravated Robbery,0,Jun,pre_covid
...,...,...,...,...,...,...,...,...,...,...
62825,2023-07-01,1,Female,Middle-aged-young,Court action,Pacific Island,Illegal Use of a Motor Vehicle,0,Jul,post_covid
62826,2023-07-01,2,Female,Middle-aged-young,Court action,Pacific Island,Illegal Use of a Motor Vehicle,0,Jul,post_covid
62827,2023-07-01,1,Female,Middle-aged-old,Court action,Asian,Abduction and Kidnapping,0,Jul,post_covid
62828,2023-07-01,1,Female,Middle-aged-young,Court action,European,Abduction and Kidnapping,0,Jul,post_covid


In [29]:
lst = []
for i in range(len(comp_df)):
    lst.append(list(comp_df.iloc[i,[2,3,5,6,8,9]]))

lst[:5]

[['Female',
  'Middle-aged-young',
  'Maori',
  'Theft (Except Motor Vehicles), N.E.C.',
  'Jun',
  'pre_covid'],
 ['Male',
  'Middle-aged-young',
  'European',
  'Theft (Except Motor Vehicles), N.E.C.',
  'Jun',
  'pre_covid'],
 ['Male',
  'Middle-aged-old',
  'Pacific Island',
  'Serious Assault Resulting in Injury',
  'Jun',
  'pre_covid'],
 ['Male',
  'Elderly',
  'European',
  'Illegal Use of a Motor Vehicle',
  'Jun',
  'pre_covid'],
 ['Male',
  'Middle-aged-young',
  'European',
  'Aggravated Robbery',
  'Jun',
  'pre_covid']]

In [30]:
te = TransactionEncoder()
te_ary = te.fit(lst).transform(lst)
new_df = pd.DataFrame(te_ary, columns=te.columns_)
new_df

,Abduction and Kidnapping,Aggravated Robbery,Aggravated Sexual Assault,Apr,Asian,Aug,Blackmail and Extortion,Common Assault,Dec,Elderly,...,Teenagers,"Theft (Except Motor Vehicles), N.E.C.",Theft From Retail Premises,Theft From a Person (Excluding By Force),Theft of Motor Vehicle Parts or Contents,Theft of a Motor Vehicle,Unknown,during_covid,post_covid,pre_covid
0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62825,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
62826,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
62827,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
62828,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [46]:
apri = apriori(new_df, min_support=0.025, use_colnames=True).sort_values('support',ascending=False)
apri

,support,itemsets
34,0.639933,(pre_covid)
16,0.505268,(Male)
10,0.494732,(Female)
21,0.455961,(Middle-aged-young)
8,0.423572,(European)
...,...,...
323,0.025402,"(Male, Middle-aged-young, Theft (Except Motor ..."
66,0.025259,"(Dec, Middle-aged-old)"
357,0.025227,"(Male, European, Illegal Use of a Motor Vehicl..."
252,0.025099,"(Male, Teenagers, European)"


In [32]:
rules = association_rules(apri, metric="lift", min_threshold=1)
rules = rules[(rules['confidence'] > 0.5)&(rules['antecedents'].apply(lambda x:len(x)>=2))]
rules.sort_values(by='confidence', inplace=True,ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
350,"(Theft (Except Motor Vehicles), N.E.C., European)",(pre_covid),0.059016,0.639933,0.043069,0.729773,1.140390,0.005302,1.332463,0.130828
656,"(Middle-aged-young, Theft (Except Motor Vehicl...",(pre_covid),0.049658,0.639933,0.035938,0.723718,1.130927,0.004161,1.303258,0.121819
602,"(Male, Theft (Except Motor Vehicles), N.E.C.)",(pre_covid),0.057234,0.639933,0.041334,0.722191,1.128542,0.004708,1.296096,0.120816
496,"(Female, Theft (Except Motor Vehicles), N.E.C.)",(pre_covid),0.051902,0.639933,0.037084,0.714505,1.116530,0.003870,1.261201,0.110082
214,"(Female, Common Assault)",(pre_covid),0.068295,0.639933,0.048353,0.707993,1.106355,0.004648,1.233078,0.103178
...,...,...,...,...,...,...,...,...,...,...
455,"(pre_covid, Oct)",(Female),0.056008,0.494732,0.028251,0.504405,1.019552,0.000542,1.019518,0.020315
254,"(Dec, pre_covid)",(Female),0.056597,0.494732,0.028426,0.502250,1.015196,0.000425,1.015104,0.015866
193,"(Aug, pre_covid)",(Female),0.057520,0.494732,0.028840,0.501384,1.013445,0.000383,1.013340,0.014076
450,"(Nov, pre_covid)",(Female),0.058237,0.494732,0.029158,0.500683,1.012030,0.000347,1.011919,0.012622


In [33]:
new_lst = []
for i in range(rules.shape[0]):
    lst = list(rules.loc[:,'antecedents'])[i]
    for j in lst:
        if np.isin(j, df.loc[:,'ANZSOC.Group']):
            new_lst.append(i)

In [34]:
new_rules = rules.iloc[new_lst,:]
warnings.filterwarnings("ignore", category=FutureWarning)

tempt_lst = []
for i in range(len(new_rules)):
    tempt_lst.append(list(new_rules.iloc[i,0])+list(new_rules.iloc[i,1]))
    
unique_list = list(set(tuple(sorted(sublist)) for sublist in tempt_lst))
unique_list = [list(sublist) for sublist in unique_list]

for i in unique_list:
    final_dic = {}
    temp_lst = []
    for j in i: 
        for col in comp_df.columns:
            if j in comp_df[col].values:
                final_dic[col] = j
    temp_lst.append(final_dic)
    for h in temp_lst:
        print(f'When "{list(h.keys())[0]}" is "{list(h.values())[0]}" and "{list(h.keys())[1]}" is "{list(h.values())[1]}" then "{list(h.keys())[2]}" is likely to be "{list(h.values())[2]}"')
    print()

When "ANZSOC.Group" is "Common Assault" and "Ethnic.Group" is "Maori" then "covid" is likely to be "pre_covid"

When "ANZSOC.Group" is "Common Assault" and "SEX" is "Male" then "Age.Group.5Yr.Band" is likely to be "Middle-aged-young"

When "SEX" is "Female" and "ANZSOC.Group" is "Serious Assault Not Resulting in Injury" then "covid" is likely to be "pre_covid"

When "Ethnic.Group" is "European" and "SEX" is "Female" then "ANZSOC.Group" is likely to be "Theft (Except Motor Vehicles), N.E.C."

When "Ethnic.Group" is "European" and "SEX" is "Female" then "ANZSOC.Group" is likely to be "Serious Assault Resulting in Injury"

When "Ethnic.Group" is "European" and "SEX" is "Male" then "ANZSOC.Group" is likely to be "Theft (Except Motor Vehicles), N.E.C."

When "ANZSOC.Group" is "Common Assault" and "Age.Group.5Yr.Band" is "Middle-aged-young" then "covid" is likely to be "pre_covid"

When "Ethnic.Group" is "European" and "SEX" is "Female" then "ANZSOC.Group" is likely to be "Serious Assault No

In [44]:
rules.iloc[new_lst,:].sort_values('lift',ascending = False).iloc[:10,:]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
352,"(Theft (Except Motor Vehicles), N.E.C., pre_co...",(European),0.078418,0.423572,0.043069,0.549219,1.296637,0.009853,1.278732,0.248241
322,"(Male, Theft (Except Motor Vehicles), N.E.C.)",(European),0.057234,0.423572,0.031291,0.546719,1.290735,0.007048,1.271679,0.238922
298,"(Female, Theft (Except Motor Vehicles), N.E.C.)",(European),0.051902,0.423572,0.027726,0.534192,1.261161,0.005741,1.237481,0.218416
733,"(Serious Assault Not Resulting in Injury, Midd...",(Female),0.043482,0.494732,0.026357,0.606149,1.225208,0.004845,1.282893,0.192168
384,"(Serious Assault Not Resulting in Injury, Maori)",(Female),0.052045,0.494732,0.030941,0.594495,1.201652,0.005192,1.246023,0.177026
746,"(Middle-aged-young, Serious Assault Resulting ...",(Female),0.048862,0.494732,0.028856,0.590554,1.193685,0.004682,1.234028,0.170593
227,"(Common Assault, post_covid)",(Male),0.045106,0.505268,0.027137,0.601623,1.190701,0.004346,1.241869,0.167724
477,"(Serious Assault Not Resulting in Injury, pre_...",(Female),0.097613,0.494732,0.056645,0.580303,1.172965,0.008353,1.203889,0.163411
514,"(Illegal Use of a Motor Vehicle, post_covid)",(Male),0.055738,0.505268,0.033026,0.592519,1.172681,0.004863,1.214121,0.155945
426,"(Serious Assault Not Resulting in Injury, Midd...",(Female),0.070253,0.494732,0.040745,0.579973,1.172297,0.005988,1.202942,0.158080
